# Step-by-Step Guide to Bitcoin Block Confirmation with Python

This guide serves as a self-teaching material for [Bitcoin](https://github.com/bitcoin/bitcoin) as well as [Jupyter Notebook](https://github.com/jupyter/notebook). Implementation is done in the most explicit way possible.

### Background

A Bitcoin block contains a magic number, the block size, a header, the transaction counter and the transaction list. Only the hash of the header will be used for hashing confirmation even though the transactions are put outside of the head. This is not a problem as Merkle root hash inside the header serves as a proof for the transactions in the block.

The size limit of a block is 1 megabyte (takes roughly 4000 transactions) and a block takes, on average, 10 minutes to be created with a right 'nonce' value.

The confirmation of a transaction depends on the number of blocks which contain that transaction (6 being the recommended minimum). These transactions must have the right 'nonce' value at the end of their headers to be able to confirm a transaction. 'Nonce' is a number that makes the hash of the header smaller than the hash 'target' which is also included in the header. Some people simply refer to the number of zeroes at the beginning of the hash to refer the difficulty of a certain block. 

Our aim is to confirm the hash of a block header. And then find the 'nonce' of a fictional block. Please note that confirming a real Bitcoin block on a generic laptop has become next to impossible (worst-case: 90+ years) due to the increasing difficulty. Our fictional block will contain a higher target to make calculation period shorter.

### Content of a block header

The header totals up to 80 bytes and consists of six parts . These are;

Block version number (4-byte integer) <br/>
Previous block hash (32-byte hex string) <br/>
Merkle root hash (32-byte hex string) <br/>
Time in seconds (4-byte integer) <br/>
Current target (4-byte integer) <br/>
Nonce (4-byte integer) 

Below variables are set with the information from [genesis block](https://en.bitcoin.it/wiki/Genesis_block) (first block ever created)

In [20]:
from hashlib import sha256
import binascii


version = 1
prev_block_hash = "0000000000000000000000000000000000000000000000000000000000000000" # Zeroes since there is no previous block
merkle_root = "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b" # Hash of all transactions related to this block
time = 1231006505 # 3 Jan 2009
current_target = 486604799 # This is decimal, but might be shown in hexadecimal in some sources.
nonce = 2083236893 # The value calculated by Satoshi Nakamoto for this block



### Little endian transformation

In [21]:
#return data in hex format 
#it just reverses the string and then swap the places of 2 bytes (ABCD12 becomes 12CDAB).
def hex_endian(string_hex):
    rev_str = string_hex[::-1]
    result = ""
    for i in range(0, len(rev_str), 2):
        result += rev_str[i+1] + rev_str[i]
    return result


#return data in hex format
#when input is 1 it gradually turns into > 0x01 > 01 > 000000 + 01 > 00000001 > "01000000"
def int_endian(integer_raw):
    integer_hex_unpadded = hex(integer_raw)[2::]
    integer_hex = (8 - len(integer_hex_unpadded)) * "0" + integer_hex_unpadded 
    return hex_endian(integer_hex)


header = (int_endian(version) + hex_endian(prev_block_hash) + hex_endian(merkle_root) + int_endian(time) \
          + int_endian(current_target) + int_endian(nonce))



### SHA256 applied twice

In [22]:
print("Full header is :\n" + header)
header_bin = binascii.unhexlify(header) # header.decode('hex') in python2

first_hash_bin = sha256(header_bin).digest()
reversed_final_hash_hex = sha256(first_hash_bin).hexdigest()
final_hash_hex = hex_endian(reversed_final_hash_hex)


Full header is :
0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c


### Hash compared to the target value


Bitcoin target number uses an very compact notation. First two digits in the hexadecimal form gives the number of zeroes on the right. This is then attached to the remaining 6 digits and then the number is padded with zeroes on the left side until it has 64 digits. The function "convert_target" takes "target" in the integer format because our data source provide this way.
            

In [23]:
#convert target from compact form to full form
def convert_target(target):
    target_hex = hex(target)[2:]
    right_padding = 2 * int(target_hex[:2], 16) - 6
    left_padding = 64 - 6 - right_padding
    result = left_padding * "0" + target_hex[2:8] + right_padding * "0"
    
    print("Resulting hash : " + final_hash_hex)
    print("Target         : " + result)
    return result

if(int(final_hash_hex, 16) < int(convert_target(current_target), 16 )):
    print("The block is confirmed.")
else:
    print("The block cannot be confirmed.")
    

Resulting hash : 000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
Target         : 00000000ffff0000000000000000000000000000000000000000000000000000
The block is confirmed.


Hash of the block is lower than the target. With this, our result is confirmed.